In [ ]:
import numpy as np
from scipy import signal

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = 0.5*fs
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
def butter_passband_filter(data, lowcut, highcut, fs, order):
    nyq = 0.5*fs
    normal_lowcut = lowcut / nyq
    normal_highcut = highcut / nyq
    b, a = signal.butter(order, [normal_lowcut, normal_highcut], btype='band', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
def logEnergy(sig,fs):
    logE=-1e30
    if len(sig)>0:
        sig2=np.power(sig,2)
        sumsig2 = np.sum(sig2)/len(sig2)
        logE=10*np.log10(sumsig2)
    return logE

In [ ]:
def mu_ener_var(sig):
    feat_mu = np.mean(sig)
    feat = np.power(sig,2)
    feat_energy = np.sum(np.absolute(feat))
    feat_variance = np.var(sig)
    return np.hstack([feat_mu, feat_energy, feat_variance])

In [ ]:
def diffe(sig):
    sig = np.delete(sig, [14,15,18,23], axis=0)
    diferencias = []
    for i in range(int(len(sig)/2)):
        diferencias.append(sig[i]-sig[i+14])
    return np.vstack(diferencias)